# General application of dRFEtools, version 0.3.0+

In [1]:
import numpy as np
import pandas as pd
import os,errno,dRFEtools
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import normalized_mutual_info_score as nmi

In [2]:
dRFEtools.__version__ 

'0.3.4'

## Define functions to analyze cross-validation

### Function

In [3]:
def dynamicRFE(estimator, x_train, x_test, y_train, y_test, fold, outdir, RF):
    # Extract feature names / generate unique features names for ranking
    features = ["feature_%d" % x for x in range(x_train.shape[1])]
    if RF:
        # Run dynamic RFE for random forest classification
        d, pfirst = dRFEtools.rf_rfe(estimator, x_train, y_train, np.array(features),
                                     fold, outdir, RANK=False) ## Using default values
    else:
        # Run dynamic RFE for all other models
        d, pfirst = dRFEtools.dev_rfe(estimator, x_train, y_train, np.array(features),
                                     fold, outdir, RANK=False) ## Do not rank
    df_elim = pd.DataFrame([{'fold':fold, 'elimination': 0.2, 'n features':k,
                             'NMI score':d[k][1], 'Accuracy score':d[k][2],
                             'ROC AUC score':d[k][3]} for k in d.keys()])
    n_features_max = max(d, key=lambda x: d[x][1])
    try:
        ## Max features from lowess curve
        n_features, _ = dRFEtools.extract_max_lowess(d) ## Using default value: 0.3
        n_redundant, _ = dRFEtools.extract_peripheral_lowess(d) ## Using default values
    except ValueError:
        ## For errors in lowess estimate
        n_features = n_features_max
        n_redundant = n_features
    ## Fit model
    estimator.fit(x_train, y_train)
    all_fts = estimator.predict(x_test)
    estimator.fit(x_train[:, d[n_redundant][4]], y_train)
    labels_pred_redundant = estimator.predict(x_test[:, d[n_redundant][4]])
    estimator.fit(x_train[:,d[n_features][4]], y_train)
    labels_pred = estimator.predict(x_test[:, d[n_features][4]])
    ## Output test predictions
    kwargs = {"average": "weighted"}
    pd.DataFrame({'fold': fold, "elimination": 0.2, 'real': y_test, 
                  'predict_all': all_fts, 'predict_max': labels_pred,
                  'predict_redundant': labels_pred_redundant})\
      .to_csv("%s/test_predictions.txt" % outdir, sep='\t', mode='a',
              index=True, header=True if fold == 0 else False)
    ## Save results into dictionary for easy manipulation
    output = dict()
    output['fold'] = fold
    output['elimination'] = 0.2
    output['n_features'] = n_features
    output['n_redundant'] = n_redundant
    output['n_max'] = n_features_max
    if RF:
        output['train_nmi'] = dRFEtools.oob_score_nmi(estimator, y_train)
        output['train_acc'] = dRFEtools.oob_score_accuracy(estimator, y_train)
        output['train_roc'] = dRFEtools.oob_score_roc(estimator, y_train)
    else:
        output['train_nmi'] = dRFEtools.dev_score_nmi(estimator,
                                                      x_train[:,d[n_features][4]],y_train)
        output['train_acc'] = dRFEtools.dev_score_accuracy(estimator,
                                                           x_train[:,d[n_features][4]],
                                                           y_train)
        output['train_roc'] = dRFEtools.dev_score_roc(estimator,
                                                      x_train[:,d[n_features][4]],y_train)
    output['test_nmi'] = nmi(y_test, labels_pred, average_method="arithmetic")
    output['test_acc'] = accuracy_score(y_test, labels_pred)
    output['test_roc'] = roc_auc_score(y_test, labels_pred, **kwargs)
    metrics_df = pd.DataFrame.from_records(output, index=[0])\
                             .reset_index().drop('index', axis=1)
    return df_elim, metrics_df

### Details on the main functions used

In [4]:
help(dRFEtools.rf_rfe)

Help on function rf_rfe in module dRFEtools.dRFEtools:

rf_rfe(estimator, X, Y, features, fold, out_dir='.', elimination_rate=0.2, RANK=True)
    Runs random forest feature elimination step over iterator process.
    
    Args:
    estimator: Random forest classifier object
    X: a data frame of training data
    Y: a vector of sample labels from training data set
    features: a vector of feature names
    fold: current fold
    out_dir: output directory. default '.'
    elimination_rate: percent rate to reduce feature list. default .2
    
    Yields:
    dict: a dictionary with number of features, normalized mutual
          information score, accuracy score, auc roc curve and array of the
          indexes for features to keep



In [5]:
help(dRFEtools.extract_max_lowess)

Help on function extract_max_lowess in module dRFEtools.lowess_redundant:

extract_max_lowess(d, frac=0.3, multi=False, acc=False)
    Extract max features based on rate of change of log10
    transformed lowess fit curve.
    
    Args:
    d: Dictionary from dRFE
    frac: Fraction for lowess smoothing. Default 3/10.
    
    Yields:
    int: number of peripheral features



In [6]:
help(dRFEtools.extract_peripheral_lowess)

Help on function extract_peripheral_lowess in module dRFEtools.lowess_redundant:

extract_peripheral_lowess(d, frac=0.3, step_size=0.02, multi=False, acc=False)
    Extract peripheral features based on rate of change of log10
    transformed lowess fit curve.
    
    Args:
    d: Dictionary from dRFE
    frac: Fraction for lowess smoothing. Default 3/10.
    step_size: Rate of change step size to analyze for extraction
    (default: 0.02)
    multi: Is the target multi-class (boolean). Default False.
    classify: Is the target classification (boolean). Default True.
    acc: Use accuracy metric to optimize data (boolean). Default False.
    
    Yields:
    int: number of peripheral features



This function has been updated from the previous name **extract_redundant_lowess**!

## Generate classification simulation data
We will first generate binary classification data on the same class as large-scale omics data.

1. We will assume a sample size of 500, which would be a large number of samples for most human tissues.
2. We will use a N of 20k for features. This is approximately the number of genes in a given region after removing low expression features.
3. Finally, we will do roughly 400 total informative (1:3 informative to redundant). This is assuming 2% of genes are significant for the phenotype of interest. 

In [7]:
# Create a dataset with only 10 informative features
X, y = make_classification(
    n_samples=500, n_features=20000, n_informative=100, n_redundant=300,
    n_repeated=0, n_classes=2, n_clusters_per_class=1, random_state=13,
    shuffle=False,
)

### Initialize stratified 5-fold cross-validation

In [8]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)

## Running analysis

### Define functions

In [9]:
def mkdir_p(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [10]:
def dRFE_run(estimator, X, y, cv, outdir, RF=True):
    mkdir_p(outdir); fold = 0
    df_dict = pd.DataFrame(); output = pd.DataFrame()
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index, :], X[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
        df_elim, metrics_df = dynamicRFE(estimator, X_train, X_test, 
                                         y_train, y_test, fold, outdir, RF)
        df_dict = pd.concat([df_dict, df_elim], axis=0)
        output = pd.concat([output, metrics_df], axis=0)
        fold += 1
    df_dict.to_csv(f"{outdir}/dRFE_simulation.tsv", sep='\t', 
                   index=False, header=True)
    output.to_csv(f"{outdir}/dRFE_simulation_metrics.tsv", 
                  sep='\t', index=False, header=True)

### Logistic regression

In [11]:
from sklearn.linear_model import LogisticRegression

outdir = "lr"
clf = LogisticRegression(max_iter=1000, n_jobs=-1)
dRFE_run(clf, X, y, cv, outdir, False)

/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
pd.read_csv(f"{outdir}/dRFE_simulation_metrics.tsv", sep="\t")

,elimination,fold,n_features,n_max,n_redundant,test_acc,test_nmi,test_roc,train_acc,train_nmi,train_roc
0,0.2,0,145,58,6553,0.88,0.472161,0.879552,1.0,1.0,1.0
1,0.2,1,20000,73,1372,0.86,0.415761,0.860000,1.0,1.0,1.0
2,0.2,2,228,182,5242,0.82,0.320912,0.820000,1.0,1.0,1.0
3,0.2,3,182,73,5242,0.91,0.564205,0.910000,1.0,1.0,1.0
4,0.2,4,1097,73,5242,0.84,0.365690,0.840000,1.0,1.0,1.0


### SGD (Stochastic Gradient Descent) Classification

In [13]:
from sklearn.linear_model import SGDClassifier

outdir = "sgd_class"
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
dRFE_run(clf, X, y, cv, outdir, False)

/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: Convergenc

In [14]:
pd.read_csv(f"{outdir}/dRFE_simulation_metrics.tsv", sep="\t")

,elimination,fold,n_features,n_max,n_redundant,test_acc,test_nmi,test_roc,train_acc,train_nmi,train_roc
0,0.2,0,20000,1097,2683,0.81,0.300823,0.810524,0.9500,0.714946,0.950000
1,0.2,1,701,877,20000,0.85,0.390494,0.850000,0.9400,0.672832,0.940024
2,0.2,2,701,116,5242,0.88,0.478239,0.880000,0.9250,0.630066,0.925198
3,0.2,3,92,286,4193,0.92,0.610964,0.920000,0.9400,0.673663,0.940049
4,0.2,4,116,2683,6553,0.87,0.446329,0.870000,0.9325,0.667614,0.932736


### SVC linear kernel

In [15]:
from sklearn.svm import LinearSVC

outdir = "svc"
clf = LinearSVC(random_state=13)
dRFE_run(clf, X, y, cv, outdir, False)

/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
/home/kynon/.local/lib/python3.11/site-p

In [16]:
pd.read_csv(f"{outdir}/dRFE_simulation_metrics.tsv", sep="\t")

,elimination,fold,n_features,n_max,n_redundant,test_acc,test_nmi,test_roc,train_acc,train_nmi,train_roc
0,0.2,0,560,92,5242,0.90,0.533486,0.90036,1.0,1.0,1.0
1,0.2,1,20000,46,1372,0.85,0.390494,0.85000,1.0,1.0,1.0
2,0.2,2,228,1097,5242,0.79,0.260181,0.79000,1.0,1.0,1.0
3,0.2,3,560,1097,5242,0.91,0.569739,0.91000,1.0,1.0,1.0
4,0.2,4,3354,8192,5242,0.85,0.390494,0.85000,1.0,1.0,1.0


### Random forest classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

outdir = "rf_class"
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, oob_score=True, random_state=13)
dRFE_run(clf, X, y, cv, outdir, True)

In [18]:
pd.read_csv(f"{outdir}/dRFE_simulation_metrics.tsv", sep="\t")

,elimination,fold,n_features,n_max,n_redundant,test_acc,test_nmi,test_roc,train_acc,train_nmi,train_roc
0,0.2,0,92,58,286,0.82,0.324322,0.820728,0.8225,0.325818,0.822500
1,0.2,1,92,145,358,0.80,0.278884,0.800000,0.8350,0.354173,0.835046
2,0.2,2,116,58,560,0.86,0.421817,0.860000,0.8400,0.369307,0.839821
3,0.2,3,116,145,228,0.83,0.344766,0.830000,0.8600,0.416094,0.859946
4,0.2,4,116,116,286,0.81,0.298752,0.810000,0.8800,0.471719,0.880072


## Session information

In [19]:
import session_info
session_info.show()